In [1]:
# Install torch geometric
import os
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.3.1%2Bcu118.html
  !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.3.1%2Bcu118.html
  !pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.3.1%2Bcu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.3.1%2Bcu118.html


In [2]:
import torch_geometric
torch_geometric.__version__

'2.4.0'

In [3]:
import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

class GNNStack(torch.nn.Module):                                                   
    def __init__(self, input_dim, hidden_dim, output_dim, args, emb=False):        
        super(GNNStack, self).__init__()                                            
        conv_model = self.build_conv_model(args.model_type)                             
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model(args.heads * hidden_dim, hidden_dim))


        self.post_mp = nn.Sequential(                                                 
            nn.Linear(args.heads * hidden_dim, hidden_dim), nn.Dropout(args.dropout),
            nn.Linear(hidden_dim, output_dim))

        self.dropout = args.dropout
        self.num_layers = args.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage
        elif model_type == 'GAT':
            return GAT
        
    def forward(self,data):
        #torch.autograd.set_detect_anomaly(True)
        x, pos_edges, neg_edges= data.x, data.edge_index , data.neg_edges
        for i in range(self.num_layers):
            x = self.convs[i](x,pos_edges,neg_edges)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout,training=self.training)
            
        x=self.post_mp(x)
        
        if self.emb==True:
            return x
        return F.log_softmax(x, dim=1)
    
    def get_node_embeddings(self, x):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=False)  
        return x
    
    def loss(self, pred, label):
        return F.binary_cross_entropy_with_logits(pred, label)

In [4]:
class GraphSage(MessagePassing):
    
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(GraphSage, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

        self.lin_l = None
        self.lin_r = None

        self.lin_l = nn.Linear(self.in_channels, self.out_channels, bias=True)
        self.lin_r = nn.Linear(self.in_channels, self.out_channels, bias=True)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    def forward(self, x,edge_index,neg_edges,size = None):
        #torch.autograd.set_detect_anomaly(True)
        out = None
        pos=self.propagate(edge_index, x=(x, x), size=size)
        neg=self.propagate(neg_edges, x=(x, x), size=size)
        out=self.lin_l(x) + self.lin_r(pos) - self.lin_r(neg)
        if self.normalize:
            out = F.normalize(out, p=2)

        return out

    def message(self, x_j):
        out = None
        out = x_j
        return out
    
    def aggregate(self, inputs, index, dim_size = None):
        out = None
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, node_dim, dim_size=dim_size, reduce='mean')
        return out


In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
from collections import defaultdict
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union
from torch_geometric.data import Data
import scipy.sparse
from torch import Tensor
from torch.utils.dlpack import from_dlpack, to_dlpack

from torch_geometric.utils.num_nodes import maybe_num_nodes

In [6]:
pos_edges=pd.read_csv('/Users/akshaj.g/Desktop/ml/social network /FacebookRecruiting/train.csv')

In [7]:
pos_edges = torch.tensor(pos_edges.values).T
num_nodes = pos_edges.max().item()
print(num_nodes)

1862220


In [8]:
def to_scipy_sparse_matrix(
    edge_index: Tensor,
    edge_attr: Optional[Tensor] = None,
    num_nodes: Optional[int] = None,
) -> scipy.sparse.coo_matrix:
    row, col = edge_index.cpu()

    if edge_attr is None:
        edge_attr = torch.ones(row.size(0))
    else:
        edge_attr = edge_attr.view(-1).cpu()
        assert edge_attr.size(0) == row.size(0)

    N = maybe_num_nodes(edge_index, num_nodes)
    out = scipy.sparse.coo_matrix(
        (edge_attr.numpy(), (row.numpy(), col.numpy())), (N, N))
    return out

In [9]:
adj_mat=to_scipy_sparse_matrix(pos_edges)

In [10]:
from torch_geometric.utils import negative_sampling
x = torch.rand(num_nodes+1, 16)
neg_edges=negative_sampling(pos_edges)

In [11]:
FB = Data(x=x, edge_index=pos_edges)
FB.neg_edges=neg_edges

In [12]:
num_nodes = FB.x.shape[0]
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
FB.train_mask = train_mask

In [13]:
FB

Data(x=[1862221, 16], edge_index=[2, 9437519], neg_edges=[2, 9437519], train_mask=[1862221])

In [14]:
import torch.optim as optim

def build_optimizer(args, params):
    weight_decay = args.weight_decay
    filter_fn = filter(lambda p : p.requires_grad, params)
    if args.opt == 'adam':
        optimizer = optim.Adam(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'sgd':
        optimizer = optim.SGD(filter_fn, lr=args.lr, momentum=0.95, weight_decay=weight_decay)
    elif args.opt == 'rmsprop':
        optimizer = optim.RMSprop(filter_fn, lr=args.lr, weight_decay=weight_decay)
    elif args.opt == 'adagrad':
        optimizer = optim.Adagrad(filter_fn, lr=args.lr, weight_decay=weight_decay)
    if args.opt_scheduler == 'none':
        return None, optimizer
    elif args.opt_scheduler == 'step':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.opt_decay_step, gamma=args.opt_decay_rate)
    elif args.opt_scheduler == 'cos':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.opt_restart)
    return scheduler, optimizer

In [15]:
'''import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args,batch_size):

    print("Node feature task")
    print()
    #test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)
    # build model
    model = GNNStack(dataset.num_node_features, args.hidden_dim,2, 
                            args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    x_values = []
    y_values = []
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        print("Entering the 1 loop")
        model.train()
        #pos_loader = DataLoader(dataset.pos_edges, batch_size=batch_size, shuffle=True)
        #neg_loader = DataLoader(dataset.neg_edges, batch_size=batch_size, shuffle=True)
        pos_loader = dataset.edge_index
        neg_loader = dataset.neg_edges
        emb = model(dataset)
        total_loss = 0
        opt.zero_grad()
        for pos_batch, neg_batch in zip(pos_loader, neg_loader):
            pos_batch = pos_batch[train_mask[pos_batch[:, 0]] & train_mask[pos_batch[:, 1]]]
            neg_batch = neg_batch[train_mask[neg_batch[:, 0]] & train_mask[neg_batch[:, 1]]]
            print(len(pos_batch))
            if len(pos_batch) == 0 or len(neg_batch) == 0:
                print("Skipping the loop due to empty batches")
                continue
            print("Entering the 2 loop")
            pos_dot_product = torch.bmm(emb[pos_batch[:, 0]].unsqueeze(2), emb[pos_batch[:, 1]].unsqueeze(2)).squeeze()
            neg_dot_product = torch.bmm(emb[neg_batch[:, 0]].unsqueeze(2), emb[neg_batch[:, 1]].unsqueeze(2)).squeeze()
            print("pos_dot_product:", pos_dot_product)
            print("neg_dot_product:", neg_dot_product)
            pos_edge_scores = pos_dot_product
            neg_edge_scores = neg_dot_product
            print("pos_edge_scores:", pos_edge_scores)
            print("neg_edge_scores:", neg_edge_scores)
            pos_labels = torch.ones(pos_edge_scores.shape)  # Label 1 for positive edges
            neg_labels = torch.zeros(neg_edge_scores.shape)  # Label 0 for negative edges
            print("pos_labels:", pos_labels)
            print("neg_labels:", neg_labels)
            pos_loss = model.loss(pos_edge_scores, pos_labels)
            neg_loss = model.loss(neg_edge_scores, neg_labels)
            print("pos_loss:", pos_loss.item())
            print("neg_loss:", neg_loss.item())
            loss = pos_loss + neg_loss
            loss.backward()
            opt.step()
            total_loss += loss.item()
        
        losses.append(total_loss)
        x_values.append(emb[0][0].item())
        y_values.append(emb[690568][0].item())
        print(f"Epoch: {epoch+1}, Loss: {total_loss}, Embeddings: {emb}")

    plt.figure(figsize=(8, 6))
    
    # Ensure x_values and y_values have the same length
    min_length = min(len(x_values), len(y_values))
    
    plt.scatter(range(1, min_length + 1), x_values[:min_length], label='X Value')
    plt.scatter(range(1, min_length + 1), y_values[:min_length], label='Y Value')
    plt.title('Embedding Values for the First Node across Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Embedding Values')
    plt.legend()
    plt.show()
        
    return emb'''

'import time\n\nimport networkx as nx\nimport numpy as np\nimport torch\nimport torch.optim as optim\nfrom tqdm import trange\nimport pandas as pd\nimport copy\n\nfrom torch_geometric.datasets import TUDataset\nfrom torch_geometric.datasets import Planetoid\nfrom torch_geometric.data import DataLoader\n\nimport torch_geometric.nn as pyg_nn\n\nimport matplotlib.pyplot as plt\n\n\ndef train(dataset, args,batch_size):\n\n    print("Node feature task")\n    print()\n    #test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)\n    # build model\n    model = GNNStack(dataset.num_node_features, args.hidden_dim,2, \n                            args)\n    scheduler, opt = build_optimizer(args, model.parameters())\n\n    # train\n    losses = []\n    test_accs = []\n    best_acc = 0\n    best_model = None\n    x_values = []\n    y_values = []\n    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):\n        print("Entering the 1 loop")\n        model.

In [25]:
import time
import traceback

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt
#torch.autograd.set_detect_anomaly(True)

def train(dataset, args, batch_size):
    print("Node feature task")
    print()
    
    # Build model
    model = GNNStack(dataset.num_node_features, args.hidden_dim, 4, args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # Train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    x_values = []
    y_values = []

    pos_edges = dataset.edge_index.t()
    neg_edges = dataset.neg_edges.t()
    print(pos_edges.shape)

    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        print("Entering the 1 loop")
        model.train()
        emb = model(dataset)
        total_loss = 0
        opt.zero_grad()

        pos_edges_shuffle_indices = torch.randperm(pos_edges.size(0))
        pos_edges_shuffled = pos_edges[pos_edges_shuffle_indices]
        pos_edges_shuffled = pos_edges_shuffled[:len(pos_edges_shuffled)//batch_size * batch_size].view(-1, batch_size, 2)

        neg_edges_shuffle_indices = torch.randperm(neg_edges.size(0))
        neg_edges_shuffled = neg_edges[neg_edges_shuffle_indices]
        neg_edges_shuffled = neg_edges_shuffled[:len(neg_edges_shuffled)//batch_size * batch_size].view(-1, batch_size, 2)

        print(pos_edges_shuffled.shape)
        for pos_batch, neg_batch in zip(pos_edges_shuffled, neg_edges_shuffled):
            pos_batch_flat = pos_batch.view(-1)
            neg_batch_flat = neg_batch.view(-1)
            with torch.no_grad():
                pos_batch_flat = torch.where(train_mask[pos_batch_flat], torch.tensor(1), pos_batch_flat.clone())
                neg_batch_flat = torch.where(train_mask[neg_batch_flat], torch.tensor(1), neg_batch_flat.clone())


            
            print(pos_batch_flat.shape)
            pos_batch = pos_batch_flat.view(pos_batch.shape)
            neg_batch = neg_batch_flat.view(neg_batch.shape)
            
            print(pos_batch.shape)
            # Filter out edges based on train_mask
            #pos_batch_filtered = pos_batch[train_mask[pos_batch[:, 0]] & train_mask[pos_batch[:, 1]]]
            #neg_batch_filtered = neg_batch[train_mask[neg_batch[:, 0]] & train_mask[neg_batch[:, 1]]]


            if len(pos_batch) == 0 or len(neg_batch) == 0:
                print("Skipping the loop due to empty batches")
                continue

            print("Entering the 2 loop")
         
            pos_dot_product = torch.bmm(emb[pos_batch[:, 0].unsqueeze(1)], emb[pos_batch[:, 1].unsqueeze(1)].transpose(1, 2)).squeeze()
            neg_dot_product = torch.bmm(emb[neg_batch[:, 0].unsqueeze(1)], emb[neg_batch[:, 1].unsqueeze(1)].transpose(1, 2)).squeeze()
            pos_edge_scores = pos_dot_product
            neg_edge_scores = neg_dot_product
            pos_labels = torch.ones(pos_edge_scores.shape)  
            neg_labels = torch.zeros(neg_edge_scores.shape)  
            pos_loss = model.loss(pos_edge_scores, pos_labels)
            neg_loss = model.loss(neg_edge_scores, neg_labels)
            print("pos_loss:", pos_loss.item())
            print("neg_loss:", neg_loss.item())
            
            import pdb; pdb.set_trace()
            
            loss = pos_loss + neg_loss
            loss.backward(retain_graph=True)
            opt.step()
            total_loss += loss.detach().item()
        
        losses.append(total_loss)
        print(f"Epoch: {epoch+1}, Loss: {total_loss}, Embeddings: {emb}")


        
    return emb


In [26]:
class objectview:
    def __init__(self, d):
        self.__dict__ = d

In [27]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    for args in [
        {'model_type': 'GraphSage', 'dataset': 'FB', 'num_layers': 2, 'heads': 1, 'batch_size': 1000, 'hidden_dim': 16, 'dropout': 0.5, 'epochs': 50, 'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0, 'weight_decay': 5e-3, 'lr': 0.001},
    ]:
        args = objectview(args)
        for model in ['GraphSage']:
            args.model_type = model

            if args.dataset == 'FB':
                dataset = FB
            else:
                raise NotImplementedError("Unknown dataset")
            NODE_EMB = train(dataset, args,1000)

Node feature task

torch.Size([9437519, 2])


Training:   0%|                                      | 0/50 [00:00<?, ?Epochs/s]

Entering the 1 loop
torch.Size([9437, 1000, 2])
torch.Size([2000])
torch.Size([1000, 2])
Entering the 2 loop
pos_loss: 0.00040967160020954907
neg_loss: 7.808108806610107
> /var/folders/_2/2n3n9_3x6ml82z7y8mx8sgbm0000gn/T/ipykernel_63102/3147216581.py(95)train()
     93             import pdb; pdb.set_trace()
     94 
---> 95             loss = pos_loss + neg_loss
     96             loss.backward(retain_graph=True)
     97             opt.step()



ipdb>  n


> /var/folders/_2/2n3n9_3x6ml82z7y8mx8sgbm0000gn/T/ipykernel_63102/3147216581.py(96)train()
     94 
     95             loss = pos_loss + neg_loss
---> 96             loss.backward(retain_graph=True)
     97             opt.step()
     98             total_loss += loss.detach().item()



...

    [... skipped 1 hidden frame]

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [16, 4]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).
> /Users/akshaj.g/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3442)run_code()
   3440                     await eval(code_obj, self.user_global_ns, self.user_ns)
   3441                 else:
-> 3442                     exec(code_obj, self.user_global_ns, self.user_ns)
   3443             finally:
   3444                 # Reset our crash handler in place



ipdb>  n


    [... skipped 1 hidden frame]

> /Users/akshaj.g/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3445)run_code()
   3443             finally:
   3444                 # Reset our crash handler in place
-> 3445                 sys.excepthook = old_excepthook
   3446         except SystemExit as e:
   3447             if result is not None:



ipdb>  c


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [16, 4]], which is output 0 of AsStridedBackward0, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
'''import time

import networkx as nx
import numpy as np
import torch
import torch.optim as optim
from tqdm import trange
import pandas as pd
import copy

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.nn as pyg_nn

import matplotlib.pyplot as plt


def train(dataset, args):

    print("Node feature task")
    print()
    test_loader = loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=False)

    # build model
    model = GNNStack(dataset.num_node_features, args.hidden_dim,2, 
                            args)
    scheduler, opt = build_optimizer(args, model.parameters())

    # train
    losses = []
    test_accs = []
    best_acc = 0
    best_model = None
    for epoch in trange(args.epochs, desc="Training", unit="Epochs"):
        total_loss = 0
        model.train()
        for batch in loader:
            opt.zero_grad()
            pred = model(batch)
            label = batch.y
            pred = pred[batch.train_mask]
            label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        losses.append(total_loss)

        if epoch % 10 == 0:
          test_acc = test(test_loader, model)
          test_accs.append(test_acc)
          if test_acc > best_acc:
            best_acc = test_acc
            best_model = copy.deepcopy(model)
        else:
          test_accs.append(test_accs[-1])
    
    return test_accs, losses, best_model, best_acc, test_loader'''

In [ ]:
FB